In [18]:
import json
import torch
import numpy as np
from tqdm import trange
from sentence_transformers import SentenceTransformer

### [重点修改] 引入 Milvus 客户端，替代 FAISS
from pymilvus import MilvusClient, DataType

print("依赖库加载完成。请确保已运行 'pip install pymilvus'")

依赖库加载完成。请确保已运行 'pip install pymilvus'


### 加载中文 Embedding 模型
### Load Chinese Embedding Model

In [5]:
embedding_model_path = "./model/bge-large-zh-v1.5"
embedding_model = SentenceTransformer(embedding_model_path, trust_remote_code=True)

# 加载到 GPU (如果可用)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
embedding_model.to(device)
print(f"模型已加载到: {device}")

模型已加载到: cuda


### 读取 JSONL 格式的案件数据
### Read Case Data in JSONL Format

In [6]:
# 读取 JSONL 文件（每行一个案件）
cases = []
with open('./law_database/all_cases_with_id.json', 'r', encoding='utf-8') as f:
    for line in f:
        if line.strip():
            cases.append(json.loads(line))

print(f"共加载 {len(cases)} 个案件")
print("\n第一个案件示例:")
print(json.dumps(cases[0], ensure_ascii=False, indent=2))

共加载 217016 个案件

第一个案件示例:
{
  "meta": {
    "accusation": [
      "故意伤害"
    ],
    "term_of_imprisonment": {
      "imprisonment": 6,
      "death_penalty": false,
      "life_imprisonment": false
    },
    "relevant_articles": [
      "234"
    ],
    "criminals": [
      "吴某某"
    ],
    "punish_of_money": 0
  },
  "fact": "攸县人民检察院指控，2017年3月2日19时许，被告人吴某某到攸县江桥街道“窝里人”饭店找易某谈论事情时，双方发生口角，吴某某顺手拿起桌上的瓷碗向易某砸去，致使易某的脸部、耳朵等部位被划伤。后经鉴定，易某的伤情构成轻伤一级。公诉机关并提供了被害人的陈述、被告人的供述、证人的证言、伤情鉴定意见书等证据，以被告人吴某某犯××向本院提起公诉，请求依法判处。\r\n",
  "id": "case_0000001"
}


In [7]:
def format_sentence(term):
    """格式化刑期"""
    if term['death_penalty']:
        return "死刑"
    elif term['life_imprisonment']:
        return "无期徒刑"
    else:
        return f"有期徒刑{term['imprisonment']}个月"

def build_case_summary(meta):
    """构建案件摘要（用于检索）"""
    accusation = '、'.join(meta['accusation'])
    criminals = '、'.join(meta['criminals'])
    articles = '、'.join([f"第{art}条" for art in meta['relevant_articles']])
    sentence = format_sentence(meta['term_of_imprisonment'])
    
    return f"罪名：{accusation}；被告人：{criminals}；判决：{sentence}；适用法条：{articles}"

In [20]:
### [重点修改] 初始化 Milvus 数据库
# 这一步会创建一个名为 legal_assistant.db 的文件，这就是你的云原生数据库雏形
db_path = "./database/legal_assistant.db"
client = MilvusClient(db_path)

collection_name = "legal_cases"

# 检查集合是否存在，如果存在则删除重建（方便你调试，实际生产中去掉 drop_collection）
if client.has_collection(collection_name):
    client.drop_collection(collection_name)

### [重点修改] 定义数据结构 (Schema)
# 相比 FAISS，这里我们可以把 ID、向量、原始文本、摘要存放在一起！
client.create_collection(
    collection_name=collection_name,
    dimension=1024,  # 你的 bge-large-zh-v1.5 模型输出维度
    primary_field_name="id",
    id_type="string", # ID 类型为字符串
    max_length=128,   # ID 最大长度
    enable_dynamic_field=True # 允许存储其他额外字段
)

print(f"Milvus 数据库已初始化: {db_path}")
print(f"集合 '{collection_name}' 已创建，等待数据插入...")

/root/miniconda3/envs/autosurvey/lib/python3.10/site-packages/milvus_lite/__init__.py:15: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution


Milvus 数据库已初始化: ./database/legal_assistant.db
集合 'legal_cases' 已创建，等待数据插入...


### 保存数据库 JSON (对应 arxiv_paper_db.json)
### Save Database JSON

In [9]:
# 保存为与 AutoSurvey 相同的格式
database_json = {
    'legal_case_info': case_database
}

with open('./database/legal_case_db.json', 'w', encoding='utf-8') as f:
    json.dump(database_json, f, ensure_ascii=False, indent=2)

print("✓ 数据库已保存: ./database/legal_case_db.json")

✓ 数据库已保存: ./database/legal_case_db.json


### 构建案件数据库 (类似 arxiv_paper_db.json 格式)
### Build Case Database (Similar to arxiv_paper_db.json Format)

In [21]:
### [重点修改] 流式处理：批量生成向量并直接插入数据库
batch_size = 32
total_cases = len(cases)

print(f"开始处理 {total_cases} 条数据，并将直接写入 Milvus...")

# 使用 trange 显示进度条
for i in trange(0, total_cases, batch_size):
    # 1. 截取当前批次的数据
    batch_data = cases[i : i + batch_size]
    
    # 2. 准备文本列表
    batch_facts = [item['fact'].strip() for item in batch_data]
    
    # 3. 生成向量 (Embedding)
    batch_vectors = embedding_model.encode(batch_facts, normalize_embeddings=True)
    
    # 4. 组装要插入的数据
    insert_data = []
    for j, case_item in enumerate(batch_data):
        # 构建我们想要存入数据库的所有字段
        meta = case_item['meta']
        summary_text = build_case_summary(meta)
        
        row = {
            "id": case_item['id'],                # 主键
            "vector": batch_vectors[j],           # 向量字段 (Milvus默认叫 vector)
            "fact": case_item['fact'],            # 原始事实 (存入数据库，检索时直接返回)
            "summary": summary_text,              # 摘要 (存入数据库)
            "accusation": meta['accusation']      # 额外元数据，方便以后做筛选
        }
        insert_data.append(row)
    
    # 5. [核心] 插入 Milvus
    client.insert(collection_name=collection_name, data=insert_data)

print("\n✓ 数据入库完成！")
print("注意：现在你不再需要保存 .index 或 .json 映射文件了，所有数据都在 legal_assistant.db 中。")

开始处理 217016 条数据，并将直接写入 Milvus...


100%|██████████| 6782/6782 [51:49<00:00,  2.18it/s]  


✓ 数据入库完成！
注意：现在你不再需要保存 .index 或 .json 映射文件了，所有数据都在 legal_assistant.db 中。


In [ ]:
def test_search_milvus(query_text, k=3):
    """测试 Milvus 检索"""
    print(f"\n{'='*80}")
    print(f"查询: {query_text}")
    print(f"{'='*80}\n")
    
    # 1. 生成查询向量
    query_vec = embedding_model.encode([query_text], normalize_embeddings=True)
    
    # 2. [重点修改] 在 Milvus 中检索
    search_res = client.search(
        collection_name=collection_name,
        data=query_vec,
        limit=k,
        # output_fields 指定我们想让数据库返回哪些字段
        output_fields=["summary", "fact", "id"] 
    )
    
    # 3. 解析结果
    print(f"找到 {k} 个最相似案件:\n")
    for rank, hit in enumerate(search_res[0], 1):
        # hit['entity'] 包含了我们在 output_fields 里请求的数据
        entity = hit['entity'] 
        distance = hit['distance']
        
        print(f"{rank}. 相似度: {distance:.4f}")
        print(f"   案件ID: {entity['id']}")
        print(f"   摘要: {entity['summary']}")
        # 截取一下事实描述，避免太长
        print(f"   事实描述: {entity['fact'][:120]}...") 
        print("-" * 50)

# 测试
test_search_milvus("偷手机", k=3)


查询: 醉酒后打人

找到 3 个最相似案件:

1. 相似度: 0.6327
   案件ID: case_0153508
   摘要: 罪名：寻衅滋事；被告人：刘某甲；判决：有期徒刑7个月；适用法条：第293条
   事实描述: 公诉机关指控，2014年5月22日21时30分许，被告人刘某甲在北正镇后四十七村华昌屯于某甲家大门外，因喝酒多了无故喊睡觉的于某甲出来要揍于某甲。于某甲出屋后，刘某甲跳进大门里与于某甲发生撕打，后刘某甲又将前来拉仗的于某甲父母于某乙和边某某...
--------------------------------------------------
2. 相似度: 0.6269
   案件ID: case_0068304
   摘要: 罪名：故意伤害；被告人：杨某甲；判决：有期徒刑12个月；适用法条：第234条
   事实描述: 宽城满族自治县人民检察院指控，2015年11月26日19时许，被告人杨某甲与杨某乙、张某某、被害人刘某甲、刘某乙在刘某乙家东屋喝酒，期间刘某甲喝多了去西屋休息，杨某甲先后两次让杨某乙去西屋找刘某甲回酒桌，刘某甲均未回，杨某甲又让张某某去找刘...
--------------------------------------------------
3. 相似度: 0.6262
   案件ID: case_0181476
   摘要: 罪名：妨害公务；被告人：张××；判决：有期徒刑12个月；适用法条：第277条
   事实描述: 天津市南开区人民检察院指控，2016年3月12日14时许，被告人张××饮酒后倒在本市南开区芥园西道与咸阳路交口处的人行道上，天津市公安局南开分局长虹派出所巡逻民警陆军帅、祈某接到指令到该地处置。到达现场后，民警将醉酒倒地的张××扶上警车，张...
--------------------------------------------------


In [12]:
# 生成案件事实的 embeddings (用于相似案例检索)
print("\n生成案件事实 embeddings...")
fact_embeddings = get_embeddings(fact_list)
print(f"✓ 完成，shape: {fact_embeddings.shape}")


生成案件事实 embeddings...


100%|██████████| 6782/6782 [50:41<00:00,  2.23it/s]


✓ 完成，shape: (217016, 1024)


In [13]:
# 生成案件摘要的 embeddings (用于快速筛选)
print("\n生成案件摘要 embeddings...")
summary_embeddings = get_embeddings(summary_list)
print(f"✓ 完成，shape: {summary_embeddings.shape}")


生成案件摘要 embeddings...


100%|██████████| 6782/6782 [05:03<00:00, 22.34it/s]


✓ 完成，shape: (217016, 1024)


### 创建 FAISS 索引
### Create FAISS Indexes

In [14]:
# 使用内积索引 (因为向量已归一化，等价于余弦相似度)
fact_index = faiss.IndexFlatIP(fact_embeddings.shape[1])
fact_index.add(fact_embeddings)

summary_index = faiss.IndexFlatIP(summary_embeddings.shape[1])
summary_index.add(summary_embeddings)

print(f"✓ 案件事实索引: {fact_index.ntotal} 个向量")
print(f"✓ 案件摘要索引: {summary_index.ntotal} 个向量")

✓ 案件事实索引: 217016 个向量
✓ 案件摘要索引: 217016 个向量


### Save faiss-index, replacing the .bin file in the database folder.
### 向量保存到本地，替换掉database文件夹中的.bin文件

In [15]:
# 保存索引文件
faiss.write_index(fact_index, './database/case_facts.index')
print("✓ 已保存: ./database/case_facts.index")

faiss.write_index(summary_index, './database/case_summaries.index')
print("✓ 已保存: ./database/case_summaries.index")

✓ 已保存: ./database/case_facts.index
✓ 已保存: ./database/case_summaries.index


### 创建 case_id 到 index 的映射 (对应 arxivid_to_index_abs.json)
### Create case_id to index Mapping

In [16]:
# 创建映射字典: {"case_0000001": 0, "case_0000002": 1, ...}
caseid_to_index = {}

for idx, (key, case_info) in enumerate(cases_list):
    case_id = case_info['id']
    caseid_to_index[case_id] = idx

# 保存映射
with open('./database/caseid_to_index.json', 'w', encoding='utf-8') as f:
    json.dump(caseid_to_index, f, ensure_ascii=False, indent=2)

print(f"✓ 已保存: ./database/caseid_to_index.json")
print(f"\n映射示例 (前5个):")
for case_id, idx in list(caseid_to_index.items())[:5]:
    print(f"  \"{case_id}\": {idx}")

✓ 已保存: ./database/caseid_to_index.json

映射示例 (前5个):
  "case_0000001": 0
  "case_0000002": 1
  "case_0000003": 2
  "case_0000004": 3
  "case_0000005": 4


### 测试检索功能
### Test Retrieval Function

In [17]:
def test_search(query_text, k=3):
    """测试案件检索"""
    print(f"\n{'='*80}")
    print(f"查询: {query_text}")
    print(f"{'='*80}\n")
    
    # 生成查询向量
    query_vec = embedding_model.encode([query_text], normalize_embeddings=True)
    
    # 检索最相似的案件
    scores, indices = fact_index.search(query_vec, k)
    
    print(f"找到 {k} 个最相似案件:\n")
    for rank, (score, idx) in enumerate(zip(scores[0], indices[0]), 1):
        case = cases_list[idx][1]
        print(f"{rank}. 相似度: {score:.4f}")
        print(f"   案件ID: {case['id']}")
        print(f"   {case['summary']}")
        print(f"   事实描述: {case['fact'][:120]}...")
        print()

# 测试
test_search("醉酒后打人", k=3)


查询: 醉酒后打人

找到 3 个最相似案件:

1. 相似度: 0.6327
   案件ID: case_0153508
   罪名：寻衅滋事；被告人：刘某甲；判决：有期徒刑7个月；适用法条：第293条
   事实描述: 公诉机关指控，2014年5月22日21时30分许，被告人刘某甲在北正镇后四十七村华昌屯于某甲家大门外，因喝酒多了无故喊睡觉的于某甲出来要揍于某甲。于某甲出屋后，刘某甲跳进大门里与于某甲发生撕打，后刘某甲又将前来拉仗的于某甲父母于某乙和边某某...

2. 相似度: 0.6269
   案件ID: case_0068304
   罪名：故意伤害；被告人：杨某甲；判决：有期徒刑12个月；适用法条：第234条
   事实描述: 宽城满族自治县人民检察院指控，2015年11月26日19时许，被告人杨某甲与杨某乙、张某某、被害人刘某甲、刘某乙在刘某乙家东屋喝酒，期间刘某甲喝多了去西屋休息，杨某甲先后两次让杨某乙去西屋找刘某甲回酒桌，刘某甲均未回，杨某甲又让张某某去找刘...

3. 相似度: 0.6262
   案件ID: case_0181476
   罪名：妨害公务；被告人：张××；判决：有期徒刑12个月；适用法条：第277条
   事实描述: 天津市南开区人民检察院指控，2016年3月12日14时许，被告人张××饮酒后倒在本市南开区芥园西道与咸阳路交口处的人行道上，天津市公安局南开分局长虹派出所巡逻民警陆军帅、祈某接到指令到该地处置。到达现场后，民警将醉酒倒地的张××扶上警车，张...

